In [ ]:
### Pandas to read the data from its source and manipulate it in memory
import pandas as pd


### The IPython.display library allows us to embed an iFrame within this 
### notebook
from IPython.display import IFrame


### To run this with functions from TheVentureCity's GitHub repository,
### clone the repository to the Google Colaboratory runtime environment
### and then import the code. This code allows us to run pre-existing functions 
### rather than having to define them inline within the notebook
### THIS IS ONLY APPLICABLE IF YOU WANT TO ACCESS THOSE FUNCTIONS ###
from importlib.machinery import SourceFileLoader
!git clone https://github.com/theventurecity/data-toolkit.git /tmp/theventurecity
!mv /tmp/theventurecity/python/tvc_transform.py tvc_transform.py
!mv /tmp/theventurecity/python/tvc_load_colab.py tvc_load_colab.py
!rm -r /tmp/theventurecity
tvct = SourceFileLoader('tvc_transform', 'tvc_transform.py').load_module()
tvcl = SourceFileLoader('tvc_load_colab', 'tvc_load_colab.py').load_module()

fatal: destination path '/tmp/theventurecity' already exists and is not an empty directory.
mv: rename /tmp/theventurecity/python/tvc_transform.py to tvc_transform.py: No such file or directory
mv: rename /tmp/theventurecity/python/tvc_load_colab.py to tvc_load_colab.py: No such file or directory
override r--r--r-- christophertrauco/wheel for /tmp/theventurecity/.git/objects/pack/pack-117670cbd4f4c586fa2c910971398cfde1066790.pack? 

In [ ]:

# 1. Extract raw event log data from a CSV
# This example uses a data file for a sample company from our GitHub repository called ServBiz. In this step we read the data file into memory as a Pandas dataframe we name "t."


filename = 'https://raw.githubusercontent.com/theventurecity/analytics/master/data/ServBiz_transactions.csv'
t = pd.read_csv(filename)
t.tail(10)
     

In [ ]:
# 2. Transform the raw data into analysis dataframes
# 2.1 Create DAU Decorated and MAU Decorated "building block" dataframes
# Note: For a more detailed discussion about creating the DAU and DAU Decorated dataframes, complete with inline code, visit Create the DAU Decorated Data Set

# 2.1.1 Create Daily Active Users (DAU) dataframe
# The DAU dataframe aggregates all activity by user and day.


# Run the create_dau_df function and show the first ten rows of the resulting dataframe
dau = tvct.create_dau_df(t, 
                         user_id = 'client_id', 
                         activity_date = 'date', 
                         inc_amt = 'value_usd',
                         segment_col = 'segment'
                        )
dau.head(10)

In [ ]:
# 2.1.2 Calculate First Date and DAU Decorated dataframes
# The create_dau_decorated_df function calls the create_first_dt_df if no first_dt dataframe is specified


# Run the create_dau_decorated_df function and show the first ten rows of the resulting dataframe
dau_decorated = tvct.create_dau_decorated_df(dau)
dau_decorated.head(10)

In [ ]:
#2.1.3 Calculate MAU Decorated dataframe

# Run create_xau_decorated_df for MAU's (using 'month'), unsegmented
mau_decorated = tvct.create_xau_decorated_df(dau_decorated, 'month', use_segment=False)
mau_decorated.tail(10)
     
#Creating Monthly Active Users Decor

In [ ]:

# 2.2 Transform into a Growth Accounting analysis dataframe
# Note: The function called in this section is different than what appears in the Mini-Pipeline: Growth Accounting, which was condensed for brevity. The resultant dataset is therefore adjusted to write only the relevant columns to Google Sheets.


TIME_PER = 'month'
USE_SEGMENT = False
KEEP_LAST_PER = True
DATE_LIMIT = None
INCL_ZERO_INC = False

user_ga, rev_ga = tvct.create_growth_accounting_dfs(mau_decorated, 
                                                    time_period=TIME_PER, 
                                                    use_segment=USE_SEGMENT, 
                                                    keep_last_period=KEEP_LAST_PER, 
                                                    date_limit=DATE_LIMIT, 
                                                    include_zero_inc=INCL_ZERO_INC)
user_ga_with_ratios = tvct.calc_user_ga_ratios(user_ga, 
                                               time_period=TIME_PER, 
                                               use_segment=USE_SEGMENT, 
                                               growth_rate_periods=12)
user_ga_with_ratios.tail(10)

In [ ]:
# 2.3 Transform into a Cohort Analysis dataframe
# reate the MAU Cohorts dataframe, using MAU Decorated as an input


mau_cohorts = tvct.create_xau_cohort_df(mau_decorated, 'month')
mau_cohorts.tail(10)

In [ ]:
#2.4.2 Calculate the DAU Histogram

dau_hist_L28 = tvct.calc_xau_hist(dau_decorated, 
                                  'day', 
                                  dau_decorated['activity_date'].max(), 
                                  window_days=28, 
                                  use_segment=False)
dau_hist_L28
     


# to g-sheets


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
### Be sure to set this value to refer to your Google Sheets workbook
GOOGLE_SHEET_KEY = '1-XnO_eWkRwX-E1fiA2Jkbe3kJvoyoPFsdeW7vnF6zS0' 


In [ ]:
print('https://docs.google.com/spreadsheets/d/' + GOOGLE_SHEET_KEY)


In [ ]:

### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(user_ga_with_ratios, 
                           GOOGLE_SHEET_KEY, 
                           'MAU Growth Accounting', 
                           gc)
     


In [ ]:

### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(mau_cohorts, 
                           GOOGLE_SHEET_KEY, 
                           'MAU Retention by Cohort', 
                           gc)
     

In [ ]:

### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(dau_hist_L28, 
                           GOOGLE_SHEET_KEY, 
                           'DAU Histogram L28', 
                           gc)
     

In [ ]:

### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(rolling_dau_mau, 
                           GOOGLE_SHEET_KEY, 
                           'Rolling DAU/MAU', 
                           gc)
     